A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

## Project Instructions
In this project, you will use regression models to predict the number of days a customer rents DVDs for.

As with most data science projects, you will need to pre-process the data provided, in this case, a csv file called rental_info.csv. Specifically, you need to:

1. Read in the csv file rental_info.csv using pandas.
2. Create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer.
3. Create two columns of dummy variables from "special_features", which takes the value of 1 when:
    1. The value is "Deleted Scenes", storing as a column called "deleted_scenes".
    1. The value is "Behind the Scenes", storing as a column called "behind_the_scenes".
4. Make a pandas DataFrame called X containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.
5. Choose the "rental_length_days" as the target column and save it as a pandas Series called y.

Following the preprocessing you will need to:

6. Split the data into X_train, y_train, X_test, and y_test train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.
7. Set random_state to 9 whenever you use a function/method involving randomness, for example, when doing a test-train split.

Recommend a model yielding a mean squared error (MSE) less than 3 on the test set

8. Save the model you would recommend as a variable named best_model, and save its MSE on the test set as best_mse.

In [138]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 1. Read in the csv file rental_info.csv using pandas.

In [139]:
rental_info = pd.read_csv('rental_info.csv')

In [140]:
rental_info.shape

(15861, 15)

## 2. Create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer.

In [141]:
rental_info['rental_length_seconds'] = (pd.to_datetime(rental_info['return_date']) - pd.to_datetime(rental_info['rental_date'])).dt.total_seconds()
rental_info['rental_length_days'] = round(rental_info['rental_length_seconds']/3600/24, 3)

## 3. Create two columns of dummy variables from "special_features", which takes the value of 1 when:
    1. The value is "Deleted Scenes", storing as a column called "deleted_scenes".
    1. The value is "Behind the Scenes", storing as a column called "behind_the_scenes".


In [174]:
rental_info['deleted_scenes'] = pd.Series([1 if val == '{"Deleted Scenes"}' else 0 for val in rental_info['special_features']])
rental_info['behind_the_scenes'] = pd.Series([1 if val == '{"Behind the Scenes"}' else 0 for val in rental_info['special_features']])

## 4. Make a pandas DataFrame called X containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.

In [177]:
rental_info.head(2)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_seconds,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,333960.0,3.865,0,0
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,245100.0,2.837,0,0


In [179]:
pd.Series(rental_info.columns).sort_values()

8                     NC-17
9                        PG
10                    PG-13
11                        R
2                    amount
12                 amount_2
18        behind_the_scenes
17           deleted_scenes
5                    length
13                 length_2
3              release_year
0               rental_date
16       rental_length_days
15    rental_length_seconds
4               rental_rate
14            rental_rate_2
6          replacement_cost
1               return_date
7          special_features
dtype: object